In [1]:
import h2o

h2o.init(min_mem_size="12g", max_mem_size="14g")


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.1+13-LTS, mixed mode)
  Starting server from c:\users\alberta\anaconda3\envs\udemy\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Alberta\AppData\Local\Temp\tmp4v0nih1q
  JVM stdout: C:\Users\Alberta\AppData\Local\Temp\tmp4v0nih1q\h2o_Alberta_started_from_python.out
  JVM stderr: C:\Users\Alberta\AppData\Local\Temp\tmp4v0nih1q\h2o_Alberta_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,2 months and 1 day
H2O cluster name:,H2O_from_python_Alberta_cohp8q
H2O cluster total nodes:,1
H2O cluster free memory:,14 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [2]:
import pandas as pd
import numpy as np

X = pd.read_csv('Train_X_new.csv',low_memory=False)
y = pd.read_csv('Train_Y_new.csv',low_memory=False)
kaggle_test = pd.read_csv('all_test_X.csv',low_memory=False)

In [3]:
df = X.merge(right = y,on = 'ID')

df['label'] = 'train'
kaggle_test['label'] = 'score'

alldf = pd.concat([df, kaggle_test], axis=0, sort=False)

In [4]:
cat_columns = ['Customer_IfCity','IndividualCustomer','IfRebate','FKRJX','IfCredit','IsFarmer','IfPrecise','IfReceivedInAdvance','IfRollOver','Type','IfJX','T_BI_IfNonPerfLoan','T_BI_LoanUse','T_BI_ifFarmRelated','MaritalStatus','Properties','Shape','HomeCondition','Degree','PrecisionType',
               'JxType','QXType','Sex','T_BI_DanbaoMethod','Use','AssureFlag','Period',
               'GiveMethond','BusinessKind','Post','RateType','InterestMethod','PersonTrade',
               'SilverRollFlag','CredType','LoanKind','Education',
               'LoanMain_TradeClass','T_BI_LendTimes','Career', 'T_BI_IfDelay']


cat_columns = list(set(cat_columns))

In [5]:
from pandas.api.types import CategoricalDtype

In [6]:
for col in cat_columns:
    alldf[col] = alldf[col].astype(CategoricalDtype())


In [7]:
hf = h2o.H2OFrame(alldf)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [8]:
for col in cat_columns:
    hf[col] = hf[col].asfactor()

In [9]:
scoring_set = hf[hf['label'] == 'score']

scoring_set = scoring_set.drop(['ID','T_BI_IfDelay','label'])

training_set = hf[hf['label'] == 'train']

training_set = training_set.drop('label')



In [10]:
data_split = training_set.split_frame(ratios=[0.75])

In [11]:
train = data_split[0]
test = data_split[1]


In [12]:
features = train.columns
features.remove("T_BI_IfDelay")
features.remove("ID")

In [13]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [16]:
# Distributed Random Forest

drf_onall = H2ORandomForestEstimator(model_id = 'drf_onall', 
                                       ntrees = 1000, 
                                       max_depth=500, 
                                       binomial_double_trees=True,
                                       nbins=5,
                                       stopping_metric="auc")

In [17]:
# Use .train() to build the model
drf_onall.train(x = features, 
                  y = "T_BI_IfDelay", 
                  training_frame = training_set)


drf Model Build progress: |███████████████████████████████████████████████| 100%


In [18]:
print(drf_onall)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  drf_onall


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.09142706769023203
RMSE: 0.3023690918236056
LogLoss: 0.3101783824340065
Mean Per-Class Error: 0.1171849913467754
AUC: 0.9414121511529193
pr_auc: 0.9112735776809044
Gini: 0.8828243023058386
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.46718405032162913: 


,0,1,Error,Rate
0,8880.0,1070.0,0.1075,(1070.0/9950.0)
1,860.0,5888.0,0.1274,(860.0/6748.0)
Total,9740.0,6958.0,0.1156,(1930.0/16698.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4671841,0.8591858,202.0
max f2,0.2511110,0.8907549,264.0
max f0point5,0.6067904,0.8633013,162.0
max accuracy,0.5197404,0.8849563,189.0
max precision,0.9986214,1.0,0.0
max recall,0.0031775,1.0,398.0
max specificity,0.9986214,1.0,0.0
max absolute_mcc,0.4897845,0.7617626,196.0
max min_per_class_accuracy,0.4308051,0.8821106,212.0
max mean_per_class_accuracy,0.4391295,0.8828150,210.0


Gains/Lift Table: Avg response rate: 40.41 %, avg score: 40.22 %



...


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2019-01-23 22:16:39,0.022 sec,0.0,nan,nan,nan,nan,nan,nan
,2019-01-23 22:16:39,0.179 sec,1.0,0.4702504,6.7219189,0.8109429,0.2477134,2.0997521,0.2040216
,2019-01-23 22:16:40,0.311 sec,2.0,0.4616952,6.2167431,0.8175695,0.2750428,2.1131567,0.1980168
,2019-01-23 22:16:40,0.393 sec,3.0,0.4478144,5.5848711,0.8249424,0.2971956,2.1154033,0.1928348
,2019-01-23 22:16:40,0.502 sec,4.0,0.4303509,4.8699280,0.8371099,0.3186501,2.1235653,0.1859121
---,---,---,---,---,---,---,---,---,---
,2019-01-23 22:17:44,1 min 4.617 sec,829.0,0.3025338,0.3105270,0.9411606,0.9104874,2.4448761,0.1158821
,2019-01-23 22:17:48,1 min 8.625 sec,879.0,0.3024860,0.3104166,0.9412210,0.9102098,2.4448761,0.1166008
,2019-01-23 22:17:52,1 min 12.681 sec,932.0,0.3024097,0.3102641,0.9413154,0.9092084,2.4448761,0.1156426
,2019-01-23 22:17:56,1 min 16.735 sec,975.0,0.3023844,0.3102304,0.9413544,0.9108185,2.4448761,0.1154629



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
IfRollOver,1467840.0,1.0,0.2896228
T_BI_QianRate,408744.3437500,0.2784666,0.0806503
T_BI_BenchmarkRate,352530.0312500,0.2401692,0.0695585
PersonTrade,221759.6250000,0.1510789,0.0437559
LoanMain_TradeClass,204192.5781250,0.1391109,0.0402897
---,---,---,---
T_BI_IfNonPerfLoan,4643.4902344,0.0031635,0.0009162
Type,2560.6337891,0.0017445,0.0005052
T_BI_LendTimes,2348.2854004,0.0015998,0.0004633
Properties,1555.2659912,0.0010596,0.0003069



See the whole table with table.as_data_frame()



In [19]:
# Predict on scoring set
predictions = drf_onall.predict(scoring_set)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [20]:
# Change h2o back into pandas dataframe
pred_df = predictions.as_data_frame(use_pandas=True)

pred_df.count()

scoring_id = kaggle_test['ID'].copy()

scoring_id.count()

pred_df.drop(["p0","p1"], axis=1, inplace=True)

pred_df.rename(columns={'predict':'Predicted'}, inplace=True)

pred_df = pd.concat([scoring_id, pred_df], axis=1)

filepath = 'scoring_xrt_15k_2k_5bins.csv'

pred_df.to_csv(filepath, index=False)
